Install the freshest stuff.

In [1]:
!pip install wandb pillow sympy graphviz

  Using cached wandb-0.15.3-py3-none-any.whl (2.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 6.6 MB/s eta 0:00:0000:0100:01
  Using cached ann_visualizer-2.5.tar.gz (4.7 kB)
  Preparing metadata (setup.py) ... done
  Using cached graphviz-0.20.1-py3-none-any.whl (47 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached GitPython-3.1.31-py3-none-any.whl (184 kB)
  Using cached sentry_sdk-1.24.0-py2.py3-none-any.whl (206 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached pathtools-0.1.2-py3-none-any.whl
  Using cached setproctitle-1.3.2-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (31 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached gitdb-4.0.10-py3-none-any.whl (62 kB)
  Using cached smmap-5.0.0-py3-none-any.whl (24 kB)
  Created wheel for ann_visualizer: filename=ann_visualizer-2.5-py3-none-any.whl size=4167 sha256=f0d2b21e966cf3f0c1f1c0

In [2]:
!nvidia-smi
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Fri May 26 01:27:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:07:00.0 Off |                  N/A |
|  0%   42C    P8     2W / 175W |    183MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

2023-05-26 01:27:04.680707: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-26 01:27:05.100447: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  1


2023-05-26 01:27:05.574671: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-26 01:27:05.587564: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-26 01:27:05.587719: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Initialize data

In [3]:
from PIL import Image
import numpy as np

# Load the image
im = Image.open('./image.png')

# Convert the image to a numpy array
im_array = np.array(im)

# Extract the x, y values from the array
x = np.arange(im_array.shape[1])
y = np.arange(im_array.shape[0])
t = [0]

# Extract the color values from the array
r = im_array[:,:,0]
g = im_array[:,:,1]
b = im_array[:,:,2]
a = im_array[:,:,3] if im.mode == 'RGBA' else None

def generate_input_output(x, y, t, r, g, b, a):
    input_data = []
    output_data = []

    for i in range(len(y)):
        for j in range(len(x)):
            input_data.append([x[j], y[i], t[0]])
            output_data.append([r[i, j], g[i, j], b[i, j], a[i, j] if a is not None else 255])

    return np.array(input_data), np.array(output_data)

input, output = generate_input_output(x, y, t, r, g, b, a)

# Normalize the input and output data
input = input / np.array([im_array.shape[1], im_array.shape[0], 1])
output = output / 255
input = np.expand_dims(input, axis=0)
output = np.expand_dims(output, axis=0)

print(input.shape)

dataset_size = len(input)


(1, 262144, 3)


Login to WandB

In [4]:
import wandb
wandb.login()

wandb: Currently logged in as: mble. Use `wandb login --relogin` to force relogin


True

Create model

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

# Define the model architecture
model = Sequential()
model.add(LSTM(128, input_shape=(input.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(128))
model.add(Dropout(0.1))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(4, activation='linear'))

2023-05-26 01:27:07.337042: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-26 01:27:07.337254: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-26 01:27:07.337399: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Compile model

In [6]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(
    loss='mean_squared_error',
    optimizer=opt,
    metrics=['mean_squared_error']
)

Compute RAM required

In [7]:
def estimate_memory_requirements(input, output, model):
    # Calculate the memory required for the input and output data
    input_data_memory = input.size * input.itemsize
    output_data_memory = output.size * output.itemsize

    # Calculate the memory required for the model weights
    model_weights_memory = 0
    for layer in model.layers:
        layer_weights = layer.get_weights()
        for weight in layer_weights:
            model_weights_memory += weight.size * weight.itemsize

    # Calculate the total memory requirement
    total_memory = input_data_memory + output_data_memory + model_weights_memory

    return total_memory

# Estimate the memory requirements for the given input, output, and model
total_memory = estimate_memory_requirements(input, output, model)
print(f"Estimated memory requirement: {total_memory} bytes")

Estimated memory requirement: 15493776 bytes


Define showing an image

In [8]:
import numpy as np
from PIL import Image
from IPython.display import display
from tensorflow.keras.callbacks import Callback

class DisplayImage(Callback):
    def on_epoch_end(self, epoch, logs=None):
      # Use the model to predict the output for the given input data
      predicted_output = model.predict(input)

      # Reshape the predicted output to match the original image dimensions
      predicted_output = predicted_output.reshape(im_array.shape[0], im_array.shape[1], -1)

      # Convert the predicted output back to the original scale (0-255)
      predicted_output = (predicted_output * 255).astype(np.uint8)

      # Create a new image using the predicted output
      predicted_image = Image.fromarray(predicted_output, mode='RGBA')

      # Display the new image
      display(predicted_image)

display_image = DisplayImage()

WandB

In [9]:
# Import wandb
import wandb
from wandb.keras import WandbCallback

# Initialize wandb
run = wandb.init(project='Image compression', entity='mble', config={
    'batch_size': dataset_size,
    'epochs': 2500
})

Save model

In [10]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_path = "training/1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path,
                                        save_weights_only=True,
                                        verbose=1)

Train a model

In [11]:
# Train the model
model.fit(input, output, epochs=2500, batch_size=dataset_size, validation_split=0, callbacks=[display_image, WandbCallback(), checkpoint_callback])

# End WandB
run.finish()

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/2500


2023-05-26 01:27:10.087949: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-26 01:27:10.089050: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-26 01:27:10.089845: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

InternalError: Graph execution error:

Detected at node 'StatefulPartitionedCall_9' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "/home/mble/.conda/envs/tf/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/mble/.conda/envs/tf/lib/python3.11/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/mble/.conda/envs/tf/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/home/mble/.conda/envs/tf/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/home/mble/.conda/envs/tf/lib/python3.11/asyncio/base_events.py", line 607, in run_forever
      self._run_once()
    File "/home/mble/.conda/envs/tf/lib/python3.11/asyncio/base_events